<a href="https://colab.research.google.com/github/qiannzzu/VPONclassification/blob/plus7/NLP_word2vec_level1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#encoding=utf-8  
import csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from scipy.sparse import coo_matrix
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import *
from sklearn.preprocessing import LabelEncoder

**Word2vec**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

contents = pd.read_csv('/content/drive/MyDrive/word2vec/input/ckipTagger.txt', sep = '\n', header=None).to_numpy().astype(str)
labels = pd.read_csv('/content/drive/MyDrive/word2vec/input/labels.txt', sep = '\n', header=None).to_numpy().astype(str)

#print(contents.ravel()[0])
#print(labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(contents.ravel(), labels, random_state=0, test_size=0.1)

#print(x_train)

In [ ]:
import gensim
from gensim.test.utils import common_texts
from gensim.models import word2vec

model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/word2vec/input/y_360W_cbow_2D_100dim_2020v1.bin', unicode_errors='ignore', binary=True)

In [ ]:
print(len(x_train))
print(x_train[0])

157700
喜特麗 全 省 安裝 80 公分 雙 渦輪 增壓 全 不鏽鋼 燒焊 隱藏式 排油煙機 抽油煙機 JT 1835 M 喜特麗


In [ ]:
#使用Cross Entropy(又稱logloss)是最常使用於分類問題的損失函數(loss functions)

def multiclass_logloss(actual, predicted, eps=1e-15): #在kaggle上常用於計算分類問題之Loss func

    """對數損失度量（Logarithmic Loss  Metric）的多分類版本。

    :param actual: 包含actual target classes的陣列

    :param predicted: 分類預測結果矩陣, 每個類別都有一個概率

    """
    actual = actual.astype(int)
    # Convert 'actual' to a binary array if it's not already:

    if len(actual.shape) == 1:

        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))

        for i, val in enumerate(actual):

            actual2[i, val] = 1

        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
   
    return -1.0 / rows * vsota

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

def sent2vec(s):
  words = word_tokenize(s) #斷詞功能 ex"At eight o'clock on Thursday morning."->['At','eight',"o'clock",...]
  words = [w for w in words if w.isalpha()]
  if len(words) > 30:
    words = words[0:30]
  if len(words) < 30:
    words.extend(["0"]*(30-len(words)))
    
    
  M = []

  for w in words:

      try:
           M.append(model[str(w)])
      except:
           M.append(model[str(0)])

  M = np.array(M)
 
  if type(M) != np.ndarray:
      # print("\n!")
      return np.zeros(30,100)
  if M.shape != (30,100):
    print(M.shape)
  return M

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from tqdm import tqdm #顯示進度條
xtrain_w2v  = [sent2vec(x) for x in tqdm(x_train)]
# xtrain_w2v  = sent2vec(xtrain[2])
xtest_w2v  = [sent2vec(x) for x in tqdm(x_test)]

100%|██████████| 17523/17523 [00:03<00:00, 4515.62it/s]


In [ ]:
print(np.shape(xtrain_w2v))
print(type(xtrain_w2v))
print(xtrain_w2v[:1])

(157700, 30, 100)
<class 'list'>
[array([[  0.5966747,   1.9254155,   2.9950552, ...,  -6.125816 ,
          7.4699855,   2.010785 ],
       [  1.8764926,   6.59889  ,  -8.513383 , ...,   1.491173 ,
         15.018477 ,  -2.2956195],
       [ 11.532459 ,   9.224527 ,   6.3856153, ...,  -1.9438661,
         -4.4415846, -10.076724 ],
       ...,
       [ -2.3035665,  -4.41802  ,  -7.8560004, ...,  -2.469303 ,
          4.8267155,   5.79327  ],
       [ -2.3035665,  -4.41802  ,  -7.8560004, ...,  -2.469303 ,
          4.8267155,   5.79327  ],
       [ -2.3035665,  -4.41802  ,  -7.8560004, ...,  -2.469303 ,
          4.8267155,   5.79327  ]], dtype=float32)]


In [ ]:
import xgboost as xgb
from sklearn.metrics import log_loss, accuracy_score, classification_report
xtrain_w2v = np.array(xtrain_w2v)
xtrain_w2v = xtrain_w2v.reshape(xtrain_w2v.shape[0], -1)
xtest_w2v = np.array(xtest_w2v)
#print(np.shape(xtest_w2v))
xtest_w2v = xtest_w2v.reshape(xtest_w2v.shape[0], -1)
y_train = y_train.ravel()
y_test = y_test.ravel()
print(np.shape(xtrain_w2v))
print(np.shape(xtest_w2v))
print(np.shape(y_test))

(157700, 3000)
(17523, 3000)


In [ ]:
#import xgboost as xgb
#xtrain_w2v = np.array(xtrain_w2v)
#xtrain_w2v = xtrain_w2v.reshape(xtrain_w2v.shape[0], -1)

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8,

                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False, tree_method='gpu_hist', gpu_id=0) #, tree_method='gpu_hist', gpu_id=0

clf.fit(xtrain_w2v, y_train)

predictions = clf.predict_proba(xtest_w2v)

arr = np.array(y_test).astype(int) # converting list to array
print(arr)

[19 30 10 ... 19  2 31]


In [ ]:
#print(predictions.shape)
#print(predictions[:5])
#print(y_test.shape)
#print(y_train.shape)
print ("word2vec+xgboost logloss: %0.3f " % multiclass_logloss(arr, predictions))

predictions = clf.predict(xtest_w2v)
print('xgboost模型的準確度:{}'.format(accuracy_score(y_test,predictions)))

#word2vec+xgboost logloss: 8.866 
#xgboost模型的準確度:0.8696570221993951

word2vec+xgboost logloss: 8.866 
xgboost模型的準確度:0.8696570221993951


In [ ]:
#w2v+LR 100dim+max_iter 400
#模型的準確度:0.7571762826000115
#LR_w2v logloss: 15.117 

from sklearn.linear_model import LogisticRegression

LR_w2v = LogisticRegression(solver='lbfgs', max_iter=400) #400
LR_w2v.fit(xtrain_w2v, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=400)

In [ ]:
predictions = LR_w2v.predict_proba(xtest_w2v) 
#print(predictions.shape)
#print(predictions[:5])
#print(y_test.shape)
print('LR_w2v模型的準確度:{}'.format(LR_w2v.score(xtest_w2v, y_test)))
arr = np.array(y_test) # converting list to array
print ("LR_w2v logloss: %0.3f " % multiclass_logloss(arr, predictions))

LR_w2v模型的準確度:0.7571762826000115
LR_w2v logloss: 15.117 
